In [12]:
import os 
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
ACTIVELOOP_TOKEN = keyring.get_password('activeloop', 'key_for_windows')
os.environ['ACTIVELOOP_TOKEN'] = ACTIVELOOP_TOKEN

In [2]:
import yt_dlp

def download_mp4_from_youtube(url):
    # set the options for the download
    filename = 'lecunninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[exp=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }
    
    # download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

In [3]:
url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

## Whisper

In [4]:
import whisper

model = whisper.load_model('base')
result = model.transcribe('./lecunninterview.mp4')
print(result['text'])

d:\projects\github\learning\llm\venv_llm\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

 Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. I wanted to talk to you about where you've gone with so supervised learning since last week spoke. In particular, I'm interested in how it relates to large language models because the large language models really came on stream since we spoke. In fact, in your talk about JEPA, which is joint embedding predictive architecture. There you go. Thank you. You mentioned that large language models lack a world model. I wan

In [5]:
# the result is generated as raw text and can be saved to a text file
with open('text.txt', 'w') as file:
    file.write(result['text'])

# Summarization with LangChain

In [14]:
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0, )

d:\projects\github\learning\llm\venv_llm\lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
d:\projects\github\learning\llm\venv_llm\lib\site-packages\langchain_community\llms\openai.py:1086: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=["", ",", "\n"]
)

In [16]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()
    
texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

llm = OpenAI()
chain = load_summarize_chain(llm, chain_type='map_reduce')
output_summary = chain.run(docs)
wrapped_text =textwrap.fill(output_summary, width=100)
print(wrapped_text)

C:\Users\ahn28\AppData\Local\Temp\ipykernel_31928\2193269566.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()


   During an interview, AI expert Jan LeCoon discusses his new joint embedding predictive
architecture and its potential to improve large language models. He also touches on his theory of
consciousness and the possibility of AI systems exhibiting it. LeCoon and the interviewer also
discuss the significance of self-supervised learning and transformer architectures in natural
language processing, as well as the limitations of large language models in representing uncertain
predictions.


In [21]:
print(chain.llm_chain.prompt.template)

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [22]:
prompt_template = """Write a concise bullet point summary of the following:

{text}


CONCISE SUMMARY IN BULLET POINTS"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                                     input_variables=['text'])

In [23]:
chain = load_summarize_chain(llm, chain_type='stuff', prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(
    output_summary,
    width=100,
    break_long_words=False,
    replace_whitespace=False
)

print(wrapped_text)

:

- Craig Smith interviews Jan LeCoon, a prominent figure in deep learning and self-supervised
learning development
- Jan discusses his joint embedding predictive architecture, which aims to fill
gaps in large language models
- He also talks about his theory of consciousness and the potential
for AI systems to exhibit conscious features
- Self-supervised learning has revolutionized natural
language processing by pre-training transformer architectures
- Large language models use self-
supervised learning to predict the next word in a text, but struggle with uncertain predictions
-
Jan's joint embedding predictive architecture aims to address this issue and has practical
applications in various fields
- Large language models lack a world model and struggle with
representing uncertainty, making it difficult to apply to tasks like video analysis
- Jan mentions
his role as a professor at NYU and chief AI scientist at Fair, a fundamental AI research lab under
Facebook


In [25]:
chain = load_summarize_chain(llm, chain_type='refine')

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(
    output_summary,
    width=100,
    break_long_words=False,
    replace_whitespace=False
)

print(wrapped_text)


Craig Smith interviews Jan LeCoon, a leading figure in deep learning development, about his views
on self-supervised learning and large language models. Jan discusses his new joint embedding
predictive architecture and its potential to address the gap in current large language models. He
also shares his theories on consciousness and the possibility of AI systems exhibiting conscious
features. They discuss Jan's background as a professor at New York University and chief AI scientist
at Fair, a fundamental AI research lab affiliated with Facebook. Jan explains the impact of self-
supervised learning on natural language processing and the use of transformer architectures in pre-
training, which has revolutionized the field in the past few years. This technique is used in
practical applications such as content moderation on social media platforms. Jan also discusses the
challenges of using generative models in self-supervised learning and the potential for his joint
embedding predictive a

## Adding Transcripts to Deep Lake

In [26]:
import yt_dlp

def download_mp4s_from_youtube(urls, job_id):
    # This will hold the titles and authors of each download video
    video_info = []
    
    for i, url in enumerate(urls):
        # set the options for the download
        file_temp = f'./download/{job_id}_{i}.mp4'
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
            'outtmpl': file_temp,
            'quiet': True,
        }
        
        # download the video file
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            title = result.get('title', "")
            author = result.get('uploader', "")
            
        # add the title and author to our list
        video_info.append((file_temp, title, author))
    
    return video_info


In [28]:
urls = ["https://www.youtube.com/watch?v=mBjPyte2ZZo&t=78s",
    "https://www.youtube.com/watch?v=cjs7QKJNVYM",]
video_details = download_mp4s_from_youtube(urls, 1)

In [34]:
import whisper

# load the model
model = whisper.load_model("base")

# iterate through each video and transribe
results = []
for video in video_details:
    result = model.transcribe(video[0])
    results.append(result['text'])
    print(f"Transcription for {video[0]}: \n{result['text']}\n")

d:\projects\github\learning\llm\venv_llm\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=de

Transcription for ./download/1_0.mp4: 
 Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. I wanted to talk to you about where you've gone with so supervised learning since last week spoke. In particular, I'm interested in how it relates to large language models because the large language models really came on stream since we spoke. In fact, in your talk about JEPA, which is joint embedding predictive architecture. There you go. Thank you. You mentioned that large la

In [35]:
with open('text.txt', 'w+') as file:
    for result in results:
        file.write(result + "\n")

In [36]:
with open('text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

docs = [Document(page_content=t) for t in texts[:4]]

In [37]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

# create Deep Lake dataset
my_activeloop_org_id = 'ahn283'
my_activeloop_dataset_name = 'langchain_course_youtube_summarizer'
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(docs)

d:\projects\github\learning\llm\venv_llm\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.0.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
C:\Users\ahn28\AppData\Local\Temp\ipykernel_31928\3431481820.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Your Deep Lake dataset has been successfully created!


Creating 4 embeddings in 1 batches of size 4:: 100%|██████████| 1/1 [00:23<00:00, 23.37s/it]

Dataset(path='hub://ahn283/langchain_course_youtube_summarizer', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   


['45dfdae9-9f6f-11ef-90a5-e0c26439031d',
 '45dfdaea-9f6f-11ef-b05b-e0c26439031d',
 '45dfdaeb-9f6f-11ef-82dd-e0c26439031d',
 '45dfdaec-9f6f-11ef-b8de-e0c26439031d']

In [38]:
retriver = db.as_retriever()
retriver.search_kwargs['distance_metric'] = 'cos'
retriver.search_kwargs['k'] = 4

In [39]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of transcripts from a video to answer the quesion in bullet points and summarized.
If you don't know the answer, just say that you don't know, dont't try to make up an answer.


{context}

Qeustion: {question}
Summarized answer in bullet points:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=['context', 'question']
)

In [41]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriver,
    chain_type_kwargs=chain_type_kwargs
)

In [42]:
print(qa.run("Summarize the mentions of google according to their AI program"))

 
- Google uses self-supervised learning in various practical applications such as content moderation on platforms like Facebook, Google, and YouTube. 
- They also use large language models to predict the next word in a text. 
- However, there are limitations to these models as they struggle to represent uncertain predictions. 
- Google is also working on developing a joint embedding predictive architecture to address these issues and potentially move towards creating AI systems with consciousness. 
- Jan LeCoon, a professor at New York University and the chief AI scientist at Fair, has been a long time proponent of self-supervised learning and has been conducting research in this field.
